# Letter prediction for English words

### What's the next letter, given 1, 2, 3, etc preceding letters?

In [ ]:
from nltk.corpus import brown
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
import string

In [ ]:
sample_size = 10000 
words = list(set([w.lower() for w in brown.words()[:sample_size]]))
words = [word for word in words if word.isalpha()]

In [ ]:
# DataFrame width: corresponds to the longest word
df_width = len(sorted(words, key=len)[-1])

In [ ]:
# Process words into "bigrams" -- in this case letter pairs
def process_word(word, n):
    letters = list(word)
    d = []
    for i in range(len(letters) - 1):
        foo = tuple([word] + [ord('_') for n in range(n-len(word[:i+1]))]
                           + [ord(l) for l in letters[:i+1]]
                           + [ord(letters[i+1])])
        d.append(foo)
    return d       

In [ ]:
process_word('keeping', 10)

In [ ]:
all_words = []
for word in words:
    all_words.extend(process_word(word, df_width))
cols = ['source']+['feature'+str(n) for n in range(df_width)]+['target']
print(all_words[0])
print(cols)
df = pd.DataFrame(all_words, columns=cols)

In [ ]:
df.head()

In [ ]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75 

In [ ]:
train, test = df[df['is_train']==True], df[df['is_train']==False]

In [ ]:
features = df.columns[1:-3]
clf = RandomForestClassifier(n_jobs=2)
y = train['target'] # Instead of y, _ = pd.factorize(train[1])

In [ ]:
features

In [ ]:
clf.fit(train[features], y)

In [ ]:
clf.predict(test[features])

In [ ]:
test.head()

In [ ]:
preds = clf.predict(test[features])
preds[:20]

In [ ]:
ct = pd.crosstab(test['target'], preds)
ct

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
y_true = np.array(test['target'])
y_pred = preds

print(precision_recall_fscore_support(y_true, y_pred, average='macro'))
print(precision_recall_fscore_support(y_true, y_pred, average='micro'))
print(precision_recall_fscore_support(y_true, y_pred, average='weighted'))

# Can we predict the length of a word given its first letter?